<a id="top"></a>

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='color:white; background:#7a96ea; border:0' role="tab" aria-controls="home"><center><font color='white'>Comparing CNN, RNN, BERT Evaluation and Performance Metrics</font></center></h3>

* [1. Helper Functions](#1)
* [2a. [Subj] CNN With Stop-Words Removal](#2a)
* [2b. [Polarity] CNN With Stop-Words Removal](#2b)
* [3a. [Subj] RNN With Lemma](#3a)
* [3b. [Polarity] RNN Without Pre-Processing](#3b)
* [4a. [Subj] BERT Training](#4a)
* [4b. [Polarity] BERT Training](#4b)
* [5. Comparing Evaluation Metrics](#5)
* [6. Comparing Performance Metrics](#6)

In [1]:
import pandas as pd
import re
import numpy as np
import time

import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.corpus import wordnet 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K

import torch
torch.cuda.empty_cache()
from torch.utils.data import TensorDataset
from transformers import BertTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification
from transformers import AdamW, get_cosine_schedule_with_warmup

from tqdm import tqdm
from sklearn.metrics import f1_score

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lowbe\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lowbe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lowbe\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
df = pd.read_csv('../data/cleaned_subj_polar__.csv', encoding='utf-8').drop(columns='Unnamed: 0')
df['length'] = df['text'].apply(lambda x: len(x.split()))
info = dict(df.length.describe())
MAX_LEN = int(info['75%'])

In [3]:
BATCH_SIZE = 512
EPOCHS = 30
LEARNING_RATE = 0.005

subj_datadict = {
    'NEUTRAL':0,
    'OPINIONATED':1
}
pol_datadict = {
    'POSITIVE':0,
    'NEGATIVE':1
}

[Back to Top](#top)
<a id="1"></a>
<h2 style='background:#7a96ea; border:0; color:white'><center><font color = 'white'>1. Helper Functions</font><center><h2>

In [4]:
def reset_random_seeds(n=0):
    '''
    Sets all necessary seed for reproduceability.
    '''
    import os
    os.environ['PYTHONHASHSEED']=str(n)
    tf.random.set_seed(n)
    np.random.seed(n)
    torch.manual_seed(n)
reset_random_seeds()

In [5]:
def get_train_test_data(data_path, column, datadict):
    df = pd.read_csv(data_path + 'cleaned_subj_polar__.csv', encoding='utf-8')
    df = df.loc[df[column].notnull()]
    df[column + 'labels'] = df[column].apply(lambda x: datadict[x])
    
    x_train, x_test, y_train, y_test = train_test_split(df['text'], df[column+'labels'], 
                                                    test_size=0.15, random_state=1, 
                                                    stratify= df[column+'labels'])
    return list(x_train), list(x_test), list(y_train), list(y_test)

In [6]:
def word_pos(word):
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    tag = nltk.pos_tag([word])[0][1][0].upper()
    
    return tag_dict.get(tag, wordnet.NOUN)

In [7]:
def tokenize_pad(x_train, x_test, MAX_LEN, lemma=False, stem=False, stopword=False, pos_lemma=False):
    if stopword:
        x_train = [' '.join([word for word in item.split() if not word in stopwords.words()]) for item in tqdm(x_train)]
    if lemma:
        lemmatizer = WordNetLemmatizer()
        x_train = [' '.join([lemmatizer.lemmatize(word) for word in seq.split()]) for seq in tqdm(x_train)]
    if pos_lemma:
        lemmatizer = WordNetLemmatizer()
        x_train = [' '.join([lemmatizer.lemmatize(word, word_pos(word)) for word in sentence.split()]) for sentence in tqdm(x_train)]
    if stem:
        porter = PorterStemmer()
        x_train = [' '.join([porter.stem(word) for word in seq.split()]) for seq in tqdm(x_train)]

    tokenizer = Tokenizer(oov_token='<OOV>')
    tokenizer.fit_on_texts(x_train)
    
    word_index = tokenizer.word_index
    
    train_sequences = tokenizer.texts_to_sequences(x_train)
    test_sequences = tokenizer.texts_to_sequences(x_test)
    
    testing_padded = pad_sequences(test_sequences, padding='post', 
                                   maxlen=MAX_LEN, truncating='post')
    training_padded = pad_sequences(train_sequences, padding='post', 
                                   maxlen=MAX_LEN, truncating='post')
    
    return training_padded, testing_padded, word_index

In [8]:
def create_cnn(em_dims, lr):
    model = keras.Sequential([
    keras.layers.Embedding(em_dims, 30, input_length=MAX_LEN),
    
    keras.layers.Conv1D(40, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv1D(5, 3, activation='relu'),
    keras.layers.MaxPooling1D(padding='same'),
    keras.layers.Dropout(0.3),
    
    keras.layers.Flatten(),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', 
                  optimizer=keras.optimizers.Adam(learning_rate=lr), 
                  metrics=[f1score])
    return model

In [9]:
def create_rnn(em_dims, lr):
    model = keras.Sequential([
        keras.layers.Embedding(len(word_index)+1, em_dims, input_length=MAX_LEN),
        keras.layers.LSTM(100, return_sequences=True),
        keras.layers.Dropout(0.5),
        keras.layers.LSTM(100),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr),
                     loss='binary_crossentropy', metrics=[f1score])
    
    return model

In [10]:
def f1score(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [11]:
def bert_tokenizer(x_train, x_test, y_train, y_test):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    
    train_encode = tokenizer.batch_encode_plus(
        x_train,
        add_special_tokens=True,
        padding=True,
        truncation = True,
        max_length = 256,   
        return_tensors='pt'
    )

    valid_encode = tokenizer.batch_encode_plus(
        x_test,
        add_special_tokens=True,
        padding=True,
        truncation = True,
        max_length = 256,   
        return_tensors='pt'
    )
    input_ids = train_encode['input_ids']
    attention_mask = train_encode['attention_mask']
    labels = torch.tensor(y_train)

    valid_input = valid_encode['input_ids']
    valid_attention = valid_encode['attention_mask']
    valid_labels = torch.tensor(y_test)

    train_data = TensorDataset(input_ids,
                              attention_mask,
                              labels)

    valid_data = TensorDataset(valid_input,
                              valid_attention,
                              valid_labels)
    return train_data, valid_data

In [12]:
def bert_dataloader(train_data, valid_data):
    BATCH_SIZE = 8
    train_dataloader = DataLoader(train_data,
                                  sampler = RandomSampler(train_data),
                                  batch_size = BATCH_SIZE)
    valid_dataloader = DataLoader(valid_data,
                                  sampler = SequentialSampler(valid_data),
                                  batch_size = BATCH_SIZE)
    return train_dataloader, valid_dataloader

In [13]:
def create_bert():
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 2, output_attentions=False, 
                                                          output_hidden_states=False)
    return model

In [14]:
def train_bert(model, train_dataloader, valid_dataloader, optimizer, scheduler):
    vloss = []
    score = []
    time_log = []
    for epoch in tqdm(range(1, EPOCHS+1)):
        reset_random_seeds(epoch)
        start = time.time()
        model.train()

        total_loss = 0

        progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
        for batch in progress_bar:
            model.zero_grad()

            batch = tuple(b.to(device) for b in batch)
            inputs = {'input_ids' : batch[0],
                      'attention_mask': batch[1],
                      'labels' : batch[2]
                      }
            outputs = model(**inputs)
            loss = outputs[0]
            total_loss += loss.item()
            loss.backward()

            optimizer.step()
            scheduler.step()

            progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

        torch.save(model.state_dict(), f'./bert comparison/BERT_subjectivity_epoch_{epoch}.pt')

        tqdm.write(f'\nEpoch {epoch}')

        avg_training_loss = total_loss/len(train_dataloader)            
        tqdm.write(f'Training loss: {avg_training_loss}')

        val_loss, predictions, actual = evaluate(valid_dataloader)
        score_f1 = f1(predictions, actual)
        tqdm.write(f'Validation loss: {val_loss}')
        tqdm.write(f'F1 Score (Weighted): {score_f1}')
        vloss.append(val_loss)
        score.append(score_f1)
        time_log.append(time.time()-start)
    return vloss, score, time_log

In [15]:
def f1(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [16]:
def evaluate(valid_dataloader):
    model.eval()

    total_eval_loss = 0
    y_hat, y = [], []

    for batch in valid_dataloader:
        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids' : batch[0],
                'attention_mask': batch[1],
                'labels' : batch[2]
                }
        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        y_hat.append(logits)
        y.append(label_ids)

    avg_eval_loss = total_eval_loss/len(valid_dataloader) 

    y_hat = np.concatenate(y_hat, axis=0)
    y = np.concatenate(y, axis=0)
            
    return avg_eval_loss, y_hat, y

In [17]:
def acpc(preds, labels):
    code_dict = {val:key for key,val in labels_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for idx in np.unique(labels_flat):
        y_hat = preds_flat[labels_flat==idx]
        y = labels_flat[labels_flat==idx]
        print(f'Class: {code_dict[idx]}')
        print(f'Accuracy: {len(y_hat[y_hat==idx])/ len(y)}\n')

In [18]:
class timelogCallback(keras.callbacks.Callback):
    '''
    Inherits callbacks.Callback from keras. Records
    the time taken from start to end of each epoch.
    '''
    
    def on_train_begin(self, logs={}):
        self.timelog = []
    def on_epoch_begin(self, batch, logs={}):
        self.start_time = time.time()
    def on_epoch_end(self, batch, logs={}):
        self.timelog.append(time.time() - self.start_time)

In [19]:
def avgNestedLists(nested_vals):
    """
    Averages a 2-D array and returns a 1-D array of all of the columns
    averaged together, regardless of their dimensions.
    """
    
    output = []
    maximum = 0
    for lst in nested_vals:
        if len(lst) > maximum:
            maximum = len(lst)
    for index in range(maximum): # Go through each index of longest list
        temp = []
        for lst in nested_vals: # Go through each list
            if index < len(lst): # If not an index error
                temp.append(lst[index])
        output.append(np.nanmean(temp))
    return output

[Back to Top](#top)
<a id="2a"></a>
<h2 style='background:#7a96ea; border:0; color:white'><center><font color = 'white'>2a. [Subj] CNN With Stop-Words Removal</font><center><h2>

In [52]:
x_train, x_test, y_train, y_test = get_train_test_data('../data/', 'subjectivity', subj_datadict)
training_padded, testing_padded, word_index = tokenize_pad(x_train, x_test, MAX_LEN, stopword=True)
y_train = np.array(y_train)
y_test = np.array(y_test)

em_dims = len(word_index)+1

100%|██████████████████████████████████████████████████████████████████████████████| 1020/1020 [02:05<00:00,  8.13it/s]


In [53]:
checkpoint_path = "./comparison/cnn1/cnn_subj.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='f1score',
                                                 save_weights_only=True,
                                                 verbose=0)

In [54]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='f1score', factor=0.1,
                              patience=4, verbose=0)
time_log = timelogCallback()
time_list = []
history_list = []

for i in tqdm(range(10)):
    reset_random_seeds(i)
    cnn_subj = create_cnn(em_dims, LEARNING_RATE)
    history = cnn_subj.fit(training_padded, y_train, batch_size=BATCH_SIZE,
                            epochs=EPOCHS, validation_data=(testing_padded, y_test),
                           callbacks=[reduce_lr, time_log, cp_callback], verbose=0)
    history_list.append(history)
    time_list.append(time_log.timelog)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:45<00:00,  4.55s/it]


In [55]:
cnn1_val_f1score = [history.history['f1score'] for history in history_list]
cnn1_val_loss = [history.history['val_loss'] for history in history_list]

cnn1_avg_val_f1score = avgNestedLists(cnn1_val_f1score)
cnn1_avg_val_loss = avgNestedLists(cnn1_val_loss)

cnn1_avg_time = avgNestedLists(time_list)

In [56]:
cnn_dict1 = {
    'cnn1_avg_val_loss':cnn1_avg_val_loss,
    'cnn1_avg_val_f1score':cnn1_avg_val_f1score,
    'cnn1_avg_time':cnn1_avg_time,
}
cnn_subj = pd.DataFrame(cnn_dict1)
cnn_subj.to_csv('./comparison/cnn_subj.csv', index=False)

[Back to Top](#top)
<a id="2b"></a>
<h2 style='background:#7a96ea; border:0; color:white'><center><font color = 'white'>2b. [Polarity] CNN With POS-Lemma</font><center><h2>

In [57]:
x_train, x_test, y_train, y_test = get_train_test_data('../data/', 'polarity', pol_datadict)
training_padded, testing_padded, word_index = tokenize_pad(x_train, x_test, MAX_LEN, pos_lemma=True)
y_train = np.array(y_train)
y_test = np.array(y_test)

em_dims = len(word_index)+1

100%|███████████████████████████████████████████████████████████████████████████████| 510/510 [00:03<00:00, 161.88it/s]


In [58]:
checkpoint_path = "./comparison/cnn2/cnn_polar.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='f1score',
                                                 save_weights_only=True,
                                                 verbose=0)

In [59]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='f1score', factor=0.1,
                              patience=4, verbose=0)
time_log = timelogCallback()
history_list = []
time_list = []

for i in tqdm(range(10)):
    reset_random_seeds(i)
    cnn_pol = create_cnn(em_dims, LEARNING_RATE)
    history = cnn_pol.fit(training_padded, y_train, batch_size=BATCH_SIZE,
                            epochs=EPOCHS, validation_data=(testing_padded, y_test),
                           callbacks=[reduce_lr, time_log, cp_callback], verbose=0)
    history_list.append(history)
    time_list.append(time_log.timelog)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:40<00:00,  4.04s/it]


In [60]:
cnn2_val_f1score = [history.history['f1score'] for history in history_list]
cnn2_val_loss = [history.history['val_loss'] for history in history_list]

cnn2_avg_val_f1score = avgNestedLists(cnn2_val_f1score)
cnn2_avg_val_loss = avgNestedLists(cnn2_val_loss)

cnn2_avg_time = avgNestedLists(time_list)

In [61]:
cnn_dict2 = {
    'cnn2_avg_val_loss':cnn2_avg_val_loss,
    'cnn2_avg_val_f1score':cnn2_avg_val_f1score,
    'cnn2_avg_time':cnn2_avg_time,
}
cnn_subj = pd.DataFrame(cnn_dict2)
cnn_subj.to_csv('./comparison/cnn_polar.csv', index=False)

[Back to Top](#top)
<a id="3a"></a>
<h2 style='background:#7a96ea; border:0; color:white'><center><font color = 'white'>3a. [Subj] RNN With Lemma</font><center><h2>

In [62]:
x_train, x_test, y_train, y_test = get_train_test_data('../data/', 'subjectivity', subj_datadict)
training_padded, testing_padded, word_index = tokenize_pad(x_train, x_test, MAX_LEN, lemma=True)
y_train = np.array(y_train)
y_test = np.array(y_test)

em_dims = len(word_index)+1

100%|███████████████████████████████████████████████████████████████████████████| 1020/1020 [00:00<00:00, 16701.31it/s]


In [63]:
checkpoint_path = "./comparison/rnn1/rnn_subj.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='f1score',
                                                 save_weights_only=True,
                                                 verbose=0)

In [64]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='f1score', factor=0.1,
                              patience=4, verbose=0)
time_log = timelogCallback()
time_list = []
history_list = []

for i in tqdm(range(10)):
    reset_random_seeds(i)
    rnn = create_rnn(em_dims, LEARNING_RATE)
    history = rnn.fit(training_padded, y_train, batch_size=BATCH_SIZE,
                            epochs=EPOCHS, validation_data=(testing_padded, y_test),
                           callbacks=[reduce_lr, time_log, cp_callback], verbose=0)
    history_list.append(history)
    time_list.append(time_log.timelog)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [11:14<00:00, 67.45s/it]


In [65]:
rnn1_val_f1score = [history.history['f1score'] for history in history_list]
rnn1_val_loss = [history.history['val_loss'] for history in history_list]

rnn1_avg_val_f1score = avgNestedLists(rnn1_val_f1score)
rnn1_avg_val_loss = avgNestedLists(rnn1_val_loss)

rnn1_avg_time = avgNestedLists(time_list)

In [66]:
rnn_dict1 = {
    'rnn1_avg_val_loss':rnn1_avg_val_loss,
    'rnn1_avg_val_f1score':rnn1_avg_val_f1score,
    'rnn1_avg_time':rnn1_avg_time,
}
rnn_subj = pd.DataFrame(rnn_dict1)
rnn_subj.to_csv('./comparison/rnn_subj.csv', index=False)

[Back to Top](#top)
<a id="3b"></a>
<h2 style='background:#7a96ea; border:0; color:white'><center><font color = 'white'>3b. [Polarity] RNN Without Pre-Processing</font><center><h2>

In [67]:
x_train, x_test, y_train, y_test = get_train_test_data('../data/', 'polarity', pol_datadict)
training_padded, testing_padded, word_index = tokenize_pad(x_train, x_test, MAX_LEN)
y_train = np.array(y_train)
y_test = np.array(y_test)

em_dims = len(word_index)+1

In [68]:
checkpoint_path = "./comparison/rnn2/rnn_polar.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='f1score',
                                                 save_weights_only=True,
                                                 verbose=0)

In [69]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='f1score', factor=0.1,
                              patience=4, verbose=0)
time_log = timelogCallback()
time_list = []
history_list = []

for i in tqdm(range(10)):
    reset_random_seeds(i)
    rnn = create_rnn(em_dims, LEARNING_RATE)
    history = rnn.fit(training_padded, y_train, batch_size=BATCH_SIZE,
                            epochs=EPOCHS, validation_data=(testing_padded, y_test),
                           callbacks=[reduce_lr, time_log, cp_callback], verbose=0)
    history_list.append(history)
    time_list.append(time_log.timelog)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:01<00:00, 18.19s/it]


In [70]:
rnn2_val_f1score = [history.history['f1score'] for history in history_list]
rnn2_val_loss = [history.history['val_loss'] for history in history_list]

rnn2_avg_val_f1score = avgNestedLists(rnn2_val_f1score)
rnn2_avg_val_loss = avgNestedLists(rnn2_val_loss)

rnn2_avg_time = avgNestedLists(time_list)

In [71]:
rnn_dict2 = {
    'rnn2_avg_val_loss':rnn2_avg_val_loss,
    'rnn2_avg_val_f1score':rnn2_avg_val_f1score,
    'rnn2_avg_time':rnn2_avg_time,
}
rnn_subj = pd.DataFrame(rnn_dict2)
rnn_subj.to_csv('./comparison/rnn_polar.csv', index=False)

[Back to Top](#top)
<a id="4a"></a>
<h2 style='background:#7a96ea; border:0; color:white'><center><font color = 'white'>4a. [Subj] BERT Training</font><center><h2>

In [50]:
x_train, x_test, y_train, y_test = get_train_test_data('../data/', 'subjectivity', subj_datadict)
x_train, y_train = bert_tokenizer(x_train, x_test, y_train, y_test)
train_dataloader, valid_dataloader = bert_dataloader(x_train, y_train)
model = create_bert()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [51]:
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-6, weight_decay=1e-1)
EPOCHS = 10 
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [52]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [54]:
vloss1, score1, time_log1 = train_bert(model, train_dataloader, valid_dataloader, optimizer, scheduler)

Epoch 1:  51%|█████████████████████████▉                         | 65/128 [00:40<00:41,  1.51it/s, training_loss=0.160]


  0%|                                                                                           | 0/10 [01:26<?, ?it/s]


Epoch 1
Training loss: 0.44290079246275127


Epoch 2:   0%|                                                                                 | 0/128 [00:00<?, ?it/s]

Validation loss: 0.34947926972223364
F1 Score (Weighted): 0.849437714923015


...


Epoch 10:  52%|█████████████████████████▊                        | 66/128 [00:40<00:37,  1.64it/s, training_loss=0.003]


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [13:55<01:22, 82.64s/it]


Epoch 10
Training loss: 0.009688658554296126


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [13:57<00:00, 83.80s/it]

Validation loss: 0.31958796563760744
F1 Score (Weighted): 0.8943629343629342


In [55]:
print(vloss1)
print(score1)
print(time_log1)

[0.34947926972223364, 0.29963777813574544, 0.2662052283792392, 0.3217841744746851, 0.31727891909363476, 0.3411908061850978, 0.32178838267598464, 0.318509542302269, 0.31947594105630467, 0.31958796563760744]
[0.849437714923015, 0.888765294771969, 0.8888339920948616, 0.8996904024767802, 0.9109351806036616, 0.8995535714285714, 0.8943629343629342, 0.8943629343629342, 0.8943629343629342, 0.8943629343629342]
[88.56869268417358, 83.92924165725708, 88.78066825866699, 80.08972549438477, 87.07562732696533, 83.02228546142578, 81.41601872444153, 81.48126745223999, 82.09722757339478, 81.47720122337341]


In [56]:
bert_dict1 = {
    'vloss1':vloss1,
    'score1':score1,
    'time_log1':time_log1
}
bert_subj = pd.DataFrame(bert_dict1)
bert_subj.to_csv('./comparison/bert_subj.csv', index=False)

[Back to Top](#top)
<a id="4b"></a>
<h2 style='background:#7a96ea; border:0; color:white'><center><font color = 'white'>4a. [Polarity] BERT Training</font><center><h2>

In [25]:
x_train, x_test, y_train, y_test = get_train_test_data('../data/', 'polarity', pol_datadict)
x_train, y_train = bert_tokenizer(x_train, x_test, y_train, y_test)
train_dataloader, valid_dataloader = bert_dataloader(x_train, y_train)
model = create_bert()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [26]:
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-6, weight_decay=1e-1)
EPOCHS = 10 
scheduler = get_cosine_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps=len(train_dataloader)*EPOCHS)

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [28]:
vloss2, score2, time_log2 = train_bert(model, train_dataloader, valid_dataloader, optimizer, scheduler)

  0%|                                                                                           | 0/10 [00:35<?, ?it/s]


Epoch 1
Training loss: 0.47788873640820384


Epoch 2:   0%|                                                                                  | 0/64 [00:00<?, ?it/s]

Validation loss: 0.2677158551911513
F1 Score (Weighted): 0.9217488510744007



 10%|████████▎                                                                          | 1/10 [01:11<05:29, 36.65s/it]

...

Validation loss: 0.0341782421261693
F1 Score (Weighted): 0.9888875169774047



 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [06:02<00:36, 36.46s/it]


Epoch 10
Training loss: 0.006810924380260985


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [06:04<00:00, 36.40s/it]

Validation loss: 0.03411598620004952
F1 Score (Weighted): 0.9888875169774047


In [29]:
print(vloss2)
print(score2)
print(time_log2)

[0.2677158551911513, 0.06799568794667721, 0.050578278644631304, 0.044139350609232984, 0.04075307100235174, 0.03807832420958827, 0.035679974826052785, 0.03453521360643208, 0.0341782421261693, 0.03411598620004952]
[0.9217488510744007, 0.9888875169774047, 0.9888875169774047, 0.9888875169774047, 0.9888875169774047, 0.9888875169774047, 0.9888875169774047, 0.9888875169774047, 0.9888875169774047, 0.9888875169774047]
[36.645992279052734, 36.24946355819702, 36.18458819389343, 36.051456928253174, 36.47990417480469, 36.28912949562073, 36.453009843826294, 36.66315293312073, 36.48644781112671, 36.48227000236511]


In [ ]:
bert_dict2 = {
    'vloss2':vloss2,
    'score2':score2,
    'time_log2':time_log2
}
bert_subj = pd.DataFrame(bert_dict2)
bert_subj.to_csv('./comparison/bert_polar.csv', index=False)